In [21]:
import numpy as np
import pandas as pd

import bebi103
import cmdstanpy
import arviz as az

import bokeh.plotting
import bokeh.io
from bokeh.io import output_file, save
bokeh.io.output_notebook()

Loading BokehJS ...

$\dot{A}=gA$ 

$g(D)=g_{max}(D_{end}-D)/(D_{end}-D_{inflection})$ if $D>D_{inflection}$

$D(t)=\int_{t_0}^t C(A)$

Same parameters for conditions but different $A_0$

In [22]:
import numpy as np
from scipy.integrate import solve_ivp
import bokeh.plotting
import bokeh.io

# Constants for the new model
class ModelParameters:
    def __init__(self, C0=0.0, C1=0.3, D_decrease=5.0, g_max=0.1):
        self.C0 = C0
        self.C1 = C1
        self.D_decrease = D_decrease
        self.D_end = 10 + D_decrease  # Define D_end in terms of D_inflect and D_decrease
        self.g_max = g_max

# Define the Heaviside step function for g
def g(D, params):
    if D < 10:
        return params.g_max
    elif D < params.D_end:
        return (params.D_end - D) / (params.D_end - 10) * params.g_max
    return 0.0

# Define the function C(A)
def C(A, params):
    return params.C0 + params.C1 * A

# ODE system for the new model
def ode_system(t, y, params):
    A, D = y
    dA_dt = A * g(D, params)         # Differential equation for A
    dD_dt = C(A, params)             # dD/dt as per given integral
    return [dA_dt, dD_dt]

# Solve the model using initial conditions and parameters
def A_theor_model(t, A0, params, D0=0.0):
    solution = solve_ivp(
        ode_system,
        [t.min(), t.max()],
        [A0, D0],
        args=(params,),
        t_eval=t,
        method='RK45'
    )
    return solution.y[0]  # Returns A(t) solution

# Main function to simulate different realizations
def ppc(x, rng):
    params = ModelParameters(
        C0=np.abs(rng.normal(0, 0.15)),
        C1=rng.normal(0.15, 0.05),
        D_decrease=10**rng.normal(1.0, 0.2),
        g_max=10**rng.normal(-1.3, 0.3)
    )
    A0 = 10**rng.normal(0.2, 0.15)    # Initial value of A based on random normal distribution
    sigma = 0 * np.abs(rng.normal(0, 1.0))  # Set sigma to control noise level, if desired
    A_solution = A_theor_model(x, A0, params)
    return rng.normal(A_solution, sigma)

# Generate x values and compute multiple realizations
x = np.linspace(0, 96, 100)
rng = np.random.default_rng()
f_ppc = np.array([ppc(x, rng) for _ in range(100)])

# Plotting with Bokeh
p = bokeh.plotting.figure(
    x_axis_label="x",
    y_axis_label="f",
    frame_height=400,
    frame_width=400,
)

# Plot each realization
for f in f_ppc:
    p.line(x + 48, f, line_alpha=0.6)  # Add transparency for clarity

bokeh.io.show(p)


In [23]:
df = pd.read_csv('../../data/SurfaceSizes.csv')

p = bokeh.plotting.figure(
    frame_width=350,
    frame_height=250,
    x_axis_label="time in hpf",
    y_axis_label="total_area-6",
    toolbar_location='above',
)

colors = bokeh.palettes.Category10_3
items = []
for color, (genotype, sub_df) in zip(colors, df.groupby("category")):
    items.append(
        (
            bokeh.models.LegendItem(
                label=genotype,
                renderers=[
                    p.scatter(
                        sub_df["time in hpf"],
                        sub_df["total_area-6"],
                        color=color,
                    )
                ],
            )
        )
    )

p.add_layout(bokeh.models.Legend(items=items), 'right')
bokeh.io.show(p)

In [35]:
sm = cmdstanpy.CmdStanModel(stan_file='combined_fit.stan')

12:35:16 - cmdstanpy - INFO - compiling stan file /home/max/Documents/01_Code/LucasPiplines/BaysianGrowth/code/Dose_model/combined_fit.stan to exe file /home/max/Documents/01_Code/LucasPiplines/BaysianGrowth/code/Dose_model/combined_fit
12:35:23 - cmdstanpy - INFO - compiled model executable: /home/max/Documents/01_Code/LucasPiplines/BaysianGrowth/code/Dose_model/combined_fit


In [36]:
def df_to_data(df, cat, t_start=48, min_time=None):
    """
    Prepare data for a given category with sorted time points.
    
    Parameters:
        df (pd.DataFrame): Input data containing categories.
        cat (str): Category name (e.g., 'Development', 'Regeneration', 'Smoc').
        t_start (float): Starting time for posterior predictive check time points.
        min_time (float, optional): Minimum time threshold for filtering data. If provided, data points
                                    with time less than min_time are excluded.

    Returns:
        dict: Prepared data for Stan model fitting.
    """
    filtered_df = df[df['category'] == cat]
    
    # Apply minimum time filter if specified
    if min_time is not None:
        filtered_df = filtered_df[filtered_df['time in hpf'] >= min_time]
    
    t = filtered_df['time in hpf'].to_numpy()
    A = filtered_df['total_area-6'].to_numpy()

    # Sort t and A based on time
    sorted_indices = np.argsort(t)
    t = t[sorted_indices]
    A = A[sorted_indices]

    N_ppc = 100
    t_ppc = np.linspace(t_start, 144, N_ppc)  # PPC from start time to 144

    return dict(t=t, A=A, N=len(t), N_ppc=N_ppc, t_ppc=t_ppc)

# Prepare data for each category, excluding times less than 60 for 'Regeneration'
data_Dev = df_to_data(df, 'Development', t_start=48)
data_Reg = df_to_data(df, 'Regeneration', t_start=60, min_time=60)
data_Smoc = df_to_data(df, 'Smoc12', t_start=48)

# Combine data for Stan sampling
stan_data = {
    'N_Dev': data_Dev['N'],
    'N_Reg': data_Reg['N'],
    'N_Smoc': data_Smoc['N'],

    't_Dev': data_Dev['t'],
    'A_Dev': data_Dev['A'],
    't_Reg': data_Reg['t'],
    'A_Reg': data_Reg['A'],
    't_Smoc': data_Smoc['t'],
    'A_Smoc': data_Smoc['A'],

    'N_ppc': data_Dev['N_ppc'],  # Assumes same PPC size across categories
    't_ppc': data_Dev['t_ppc'],   # PPC time points, could be generalized if needed
    't_Reg_ppc': data_Reg['t_ppc']   # PPC time points, could be generalized if needed
}

In [37]:
samples = sm.sample(data=stan_data,iter_warmup=3000,iter_sampling=1000,show_console = False )
samples = az.from_cmdstanpy(posterior=samples, posterior_predictive=['A_Dev_ppc', 'A_Reg_ppc', 'A_Smoc_ppc'])
bebi103.stan.check_all_diagnostics(samples)

12:35:27 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

12:44:08 - cmdstanpy - INFO - CmdStan done processing.
12:44:08 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: Exception: ode_rk45: ode parameters and data[3] is inf, but must be finite! (in 'combined_fit.stan', line 34, column 8 to column 92) (in 'combined_fit.stan', line 110, column 4 to column 89)
	Exception: Exception: ode_rk45: ode parameters and data[3] is inf, but must be finite! (in 'combined_fit.stan', line 34, column 8 to column 92) (in 'combined_fit.stan', line 110, column 4 to column 89)
Exception: Exception: ode_rk45: initial state[1] is inf, but must be finite! (in 'combined_fit.stan', line 34, column 8 to column 92) (in 'combined_fit.stan', line 112, column 4 to column 102)
	Exception: Exception: ode_rk45: initial state[1] is inf, but must be finite! (in 'combined_fit.stan', line 34, column 8 to column 92) (in 'combined_fit.stan', line 112, column 4 to column 102)
	Exception: Exception: ode_rk45: initial state[1] is inf, but must be finite! (in 'comb


Effective sample size looks reasonable for all parameters.

Rhat looks reasonable for all parameters.

70 of 4000 (1.75%) iterations ended with a divergence.
  Try running with larger adapt_delta to remove divergences.

3 of 4000 (0.075%) iterations saturated the maximum tree depth of 10.
  Try running again with max_treedepth set to a larger value to avoid saturation.

E-BFMI indicated no pathological behavior.


12

In [38]:
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=['A_0_Dev', 'sigma', 'D_decrease', 'C0', 'C1', 'g_max'],
        xtick_label_orientation=np.pi / 4,
        plot_ecdf=True
    )
)


In [39]:
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=['A_0_Reg', 'D_decrease', 'C0', 'C1', 'g_max', 'D_0_Reg'],
        xtick_label_orientation=np.pi / 4,
        plot_ecdf=True
    )
)

In [40]:
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=['A_0_Smoc', 'sigma', 'D_decrease', 'C0_Smoc', 'C1_Smoc', 'g_max'],
        xtick_label_orientation=np.pi / 4,
        plot_ecdf=True
    )
)

In [10]:
samples

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats

In [41]:
f_ppc_Dev = samples.posterior_predictive.A_Dev_ppc.stack(
    {"sample": ("chain", "draw")}
).transpose("sample", "A_Dev_ppc_dim_0")

bokeh.io.show(
    bebi103.viz.predictive_regression(
        f_ppc_Dev,
        samples_x=np.array(data_Dev["t_ppc"]),       # PPC x-values for Development
        data=np.dstack((data_Dev["t"], data_Dev["A"])).squeeze(),  # Observed Development data
        x_axis_label='t',
        y_axis_label='A (Development)',
    )
)


In [42]:
f_ppc_Reg = samples.posterior_predictive.A_Reg_ppc.stack(
    {"sample": ("chain", "draw")}
).transpose("sample", "A_Reg_ppc_dim_0")

bokeh.io.show(
    bebi103.viz.predictive_regression(
        f_ppc_Reg,
        samples_x=np.array(data_Reg["t_ppc"]),       # PPC x-values for Development
        data=np.dstack((data_Reg["t"], data_Reg["A"])).squeeze(),  # Observed Development data
        x_axis_label='t',
        y_axis_label='A (Regeneration)',
    )
)


In [43]:
f_ppc_Smoc = samples.posterior_predictive.A_Smoc_ppc.stack(
    {"sample": ("chain", "draw")}
).transpose("sample", "A_Smoc_ppc_dim_0")

bokeh.io.show(
    bebi103.viz.predictive_regression(
        f_ppc_Smoc,
        samples_x=np.array(data_Smoc["t_ppc"]),       # PPC x-values for Development
        data=np.dstack((data_Smoc["t"], data_Smoc["A"])).squeeze(),  # Observed Development data
        x_axis_label='t',
        y_axis_label='A (Smoc)',
    )
)

In [14]:
bebi103.stan.clean_cmdstan()